In [65]:
import pandas as pd
import sys  
sys.path.insert(0, './../')
from importlib import reload
import bbdc2021 as bbdc
reload(bbdc)

<module 'bbdc2021' from './../bbdc2021.py'>

In [44]:
mainPrediction = pd.read_csv("../submissions/submission07/challenge_submission_t08n03b2.csv")
secondPrediction = pd.read_csv("../submissions/submission03/2d_PredictionChallenge_mel.csv")

In [47]:
def addMissingSequences(mainPredDf, secondPredDf, minLength=0):
    predictionsToAdd = [] 
    secondPredDf['duration'] = secondPredDf.offset - secondPredDf.onset
    for filename in secondPredDf.filename.unique():
        mainFileRows = mainPredDf.loc[mainPredDf['filename'] == filename]
        secondFileRows = secondPredDf.loc[(secondPredDf['filename'] == filename) & (secondPredDf['duration']>=minLength)]
        for index, sequence in secondFileRows.iterrows():
            overlappingSequences = mainFileRows.loc[(mainFileRows['onset']<=sequence.offset) & (mainFileRows['offset']>=sequence.onset)]
            if len(overlappingSequences)==0:
                predictionsToAdd.append([sequence.filename, sequence.onset, sequence.offset, sequence.event_label])
    additionalPredictions = pd.DataFrame(data=predictionsToAdd, columns=['filename', 'onset', 'offset', 'event_label'])
    print("Added "+str(len(predictionsToAdd)) + " sequences")
    return mainPredDf.append(additionalPredictions)
    
predictionWithAddedSequences = bbdc.addMissingSequences(mainPrediction, secondPrediction)

Added 695 sequences


In [25]:
predictionWithAddedSequences.to_csv("../predictions/jannes/predictionAlexWithAddedSequences.csv", index=False)

In [70]:
mergedPrediction = bbdc.mergeNeighboringSequences(predictionWithAddedSequences, maxSequenceLength = 2)
mergedPrediction.to_csv("../predictions/jannes/predictionAlexWithAddedSequences_merged.csv", index=False)

Merged 57 sequences
